# 实验一
自定义深度卷积申请网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [1]:
#加载ImageDataGenerator模块
from keras.preprocessing.image import ImageDataGenerator
#加载定义CNN的layers、activate function等模块
from keras.layers import Activation,Dropout,Flatten,Dense
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
#以下为将预测score转换为类别信息的to_categorical、图像预处理等函数
from keras.utils import to_categorical 
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

Using TensorFlow backend.


### 定义CNN网络结构
首先尝试：自定义深度CNN结构

然后尝试：定义Alex/VGG网络结构


In [2]:
#首先定义图像的长和宽，目的是告诉网络输入图像矩阵的大小，便于搭建网络结构
img_width,img_height = 150,150
#以下为确认图像的通道分量
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:   #通道是3
    input_shape = (img_width, img_height, 3)    #tensorflow的通道参数在最后
    
#在此定义网络结构    
model = Sequential()
#在此调用compile函数定义loss function，optimizer和metrics
model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
#像素点为148×148×32个， 3×3×32×3+32=896 因为有3×3×3×32个卷即参出需训练 有32 个偏执量（防止梯度爆炸）
#在3个通道上卷积，它的卷积参数是不一样的，即3×3×32是在一个通道上卷积，有三个通道。每一个feature maps对应
#一个偏执
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())   #flatten(使平坦)函数：返回一个一维数组。它只能用于numpy对象，即array或mat，不适用于list
#flatten就是把数组降到一维，默认按行方向降
model.add(Dense(64))   #Dense类：他是标准的一维全链接层
model.add(Activation('relu'))   #防止梯度消失，但会造成过拟合（一些不是此类的也预测为此类了）
model.add(Dropout(0.5))   #防止过拟合
model.add(Dense(1))
model.add(Activation('sigmoid'))   
                                          #compile(编译，编辑)
model.compile(loss='binary_crossentropy',   #cross-entropy:交叉熵
              optimizer='rmsprop',    #optimizer:优化器
             metrics=['accuracy'])       #metric:

### 查看model架构



In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

### 定义ImageDataGenerator


In [4]:
#指定train和validation数据路径
train_data_dir = r'/home/jiaokailang/文档/机器视觉/11.6/dogs-vs-cats/train'
validation_data_dir = r'/home/jiaokailang/文档/机器视觉/11.6/dogs-vs-cats/validation'
#指定train和validation数据规模
nb_train_samples = 10835    #sample:样本
nb_validation_samples = 4000 


epochs = 1
batch_size = 20

#声明ImageDataGenerator类型的train_datagen用于对training图像进行数据像素归一化和扩增
train_datagen = ImageDataGenerator(   #此函数：扩充图像数据集
    rescale=1. / 255,  #归一化：防止梯度爆炸    #rescale:重新调节
    shear_range=0.2,   #shear（修剪）：浮点型。变换角度
    zoom_range=0.2,   #zoom（急速移动）：随机缩放
    #图像变换，（图像扩增） 变为4张
    #训练级是为了增加多种情况。测试集不需要，因为它可能会增加测试难度
    horizontal_flip=True)   #水平翻转

#声明ImageDataGenerator类型的test_datagen用于对validation图像进行像素归一化
test_datagen = ImageDataGenerator(rescale=1. / 255)  
#调用flow_from_directory生成train_generator      ##generator:生成器，发生器
train_generator = train_datagen.flow_from_directory(   
    train_data_dir,
    target_size=(img_width, img_height),  #target_size:整数tuple，默认为（256，256）
    #color_mode:grayscale/rgb,默认为rgb，此类图片为三通道
    batch_size=batch_size,
    class_mode='binary')#该参数决定了返回的标签数组的形式，binary:返回1D的二值标签
#flow_from_directory函数：使得图片不需要标签，将目录中的图片根据首字母顺序自动贴标签（因为有binary二分类）
#此函数生成经过数据提升/归一化后的数据
#首字母在前的，为0

#调用flow_from_directory生成validation_generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')




Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [ ]:
#调用并设置fit_generator函数训练模型   fit(合适的，匹配，安装)
model.fit_generator(   
    train_generator,  #图片
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,   #验证图片
    validation_steps=nb_validation_samples // batch_size)
#fit_generator:提取图片

Epoch 1/1
195/541 [=========>....................] - ETA: 14:40 - loss: 0.7214 - accuracy: 0.5587

### 使用训练后模型预测图像





In [ ]:
import cv2
#调用opencv的imread函数和resize函数读取测试样本并进行尺寸缩放，此处路径可更换
img = cv2.resize(cv2.imread(r'/home/jiaokailang/文档/机器视觉/11.6/dogs-vs-cats/test/7.jpg'), (img_width, img_height)).astype(np.float32)

#对图像像素矩阵进行tensor类型转换
x = img_to_array(img)
x = np.expand_dims(x, axis=0)  #numpy.expand_dims:在axis上把数据加上去，即[1,2]-[[1,2]],axis=0表示不变化，axis=1
#用训练好的模型对测试样本进行预测
score = model.predict(x)
#predict:返回一个0-1之间的矩阵

print(score)